<a href="https://colab.research.google.com/github/MEYLIN84/DATA-UPC/blob/main/PYTHON_PROYECTO_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.- INTRODUCCIÓN
Las principales ciudades turísticas de Francia son Londres y París, son ciudades muy concurridas y populares para vacaciones. Su cultura diversa y ofrecen una amplia variedad de experiencias muy buscadas. Intentamos agrupar los barrios de Londres y París respectivamente y obtener información sobre cómo se ven ahora.

2.- EL PROBLEMA El objetivo es ayudar a los turistas a elegir sus destinos en función de las experiencias que los barrios tienen para ofrecer y lo que les gustaría tener. Esto también ayuda a las personas a tomar decisiones si están pensando en migrar a Londres o París o incluso si quieren reubicar barrios dentro de la ciudad. Nuestros hallazgos ayudarán a las partes interesadas a tomar decisiones informadas y abordar cualquier inquietud que tengan, incluidos los diferentes tipos de cocinas, tiendas de provisiones y lo que la ciudad tiene para ofrecer.


3.- DESCRIPCIÓN DE LA DATA Requerimos datos de ubicación geográfica tanto para Londres como para París. Los códigos postales de cada ciudad sirven como punto de partida. Usando los códigos postales que usamos podemos encontrar los vecindarios, distritos, lugares y sus categorías de lugares más populares.

3.1.- LONDRES Para derivar nuestra solución, extraemos nuestros datos de https://en.wikipedia.org/wiki/List_of_areas_of_London Esta página de wikipedia tiene información sobre todos los barrios, nos limitamos a Londres.
Municipio : Nombre del Barrio
ciudad : Nombre del municipio
post_code : Códigos postales de Londres.
Esta página de wikipedia carece de información sobre las ubicaciones geográficas. Para solucionar este problema utilizamos la API de ArcGIS

3.2.- ArcGIS API.- ArcGIS Online le permite conectar personas, ubicaciones y datos mediante mapas interactivos. Trabaje con estilos inteligentes basados ​​en datos y herramientas de análisis intuitivas que brindan inteligencia de ubicación. Comparta sus conocimientos con el mundo o con grupos específicos.

Más específicamente, usamos ArcGIS para obtener las ubicaciones geográficas de los barrios de Londres. Las siguientes columnas se agregan a nuestro conjunto de datos inicial que prepara nuestros datos.

latitude : Latitud para el vecindario
longitude : Longitud del vecindario

3.3.- PARIS Para derivar nuestra solución, aprovechamos los datos JSON disponibles en https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e El archivo JSON tiene datos sobre todos los barrios de Francia, lo limitamos a París.

postal_code : Códigos postales de Francia
nom_comm : Nombre de los barrios de Francia
nom_dept : Nombre de los distritos, equivalente a ciudades en Francia
geo_point_2d : Tupla que contiene la latitud y longitud de los Barrios.

3.4.- FOURSQUARE API DATA Necesitaremos datos sobre diferentes lugares en diferentes vecindarios de ese distrito específico. Para obtener esa información, utilizaremos la información de ubicación de "Foursquare". Foursquare es un proveedor de datos de ubicación con información sobre todo tipo de lugares y eventos dentro de un área de interés. Dicha información incluye nombres de lugares, ubicaciones, menús e incluso fotos. Como tal, la plataforma de ubicación de Foursquare se utilizará como la única fuente de datos, ya que toda la información requerida se puede obtener a través de la API.
Después de encontrar la lista de vecindarios, nos conectamos a la API de Foursquare para recopilar información sobre los lugares dentro de todos y cada uno de los vecindarios. Para cada barrio, hemos elegido que el radio sea de 500 metros. Los datos recuperados de Foursquare contenían información de lugares dentro de una distancia específica de la longitud y latitud de los códigos postales. La información obtenida por sede es la siguiente:
Barrio : Nombre del Barrio
Latitud del vecindario: Latitud del vecindario
Longitud del vecindario: Longitud del vecindario
Lugar: Nombre del lugar
Latitud del lugar: Latitud del lugar
Longitud del lugar: Longitud del lugar
Categoría del lugar: Categoría del lugar
Según toda la información recopilada tanto para Londres como para París, tenemos datos suficientes para construir nuestro modelo. Agrupamos los vecindarios en función de categorías de lugares similares. Luego presentamos nuestras observaciones y hallazgos. Con estos datos, nuestros grupos de interés pueden tomar la decisión necesaria.

4.- METODOLOGÍA

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

4.1.- ANALIZANDO LA DATA

In [2]:
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data = wiki_london_data[1]
wiki_london_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
527,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
528,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
529,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
530,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


In [10]:
!wget -q -O 'france-data.json' https://data.iledefrance.fr/explore/dataset/base-officielle-des-codes-postaux/download/?format=json&timezone=America/Bogota&lang=fr
print("Data Downloaded!")
paris_raw = pd.read_json(data)
paris_raw.head()


Data Downloaded!


,datasetid,recordid,fields,geometry,record_timestamp
0,base-officielle-des-codes-postaux,7232561e7cdbaadfd19686307de99c7983b578bd,"{'dep': '77', 'nom_commune': 'CHEVRY COSSIGNY'...","{'type': 'Point', 'coordinates': [2.6765377, 4...",2022-06-22T13:28:56.668Z
1,base-officielle-des-codes-postaux,99f8e63ce450eccd6dd0d36b6f62ecfa78b0c1f7,"{'dep': '77', 'nom_commune': 'DAMMARTIN SUR TI...","{'type': 'Point', 'coordinates': [2.903759408,...",2022-06-22T13:28:56.668Z
2,base-officielle-des-codes-postaux,e6564a927f7fbc1c55d38179899ac5272c0eb3d3,"{'dep': '77', 'nom_commune': 'NOISY RUDIGNON',...","{'type': 'Point', 'coordinates': [2.933839851,...",2022-06-22T13:28:56.668Z
3,base-officielle-des-codes-postaux,97451499e7a2e635754d0f4e91756474bd411df1,"{'dep': '77', 'nom_commune': 'ORLY SUR MORIN',...","{'type': 'Point', 'coordinates': [3.226766301,...",2022-06-22T13:28:56.668Z
4,base-officielle-des-codes-postaux,5c4d859bba162696dfad0c6348534fb9c39e53be,"{'dep': '77', 'nom_commune': 'OTHIS', 'code_po...","{'type': 'Point', 'coordinates': [2.655019992,...",2022-06-22T13:28:56.668Z


4.2 PROCESAMIENTO DE DATA

En el caso de Londres, sustituimos los espacios por guiones bajos en el título. 

In [22]:
wiki_london_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)


Para París, desglosamos cada uno de los campos anidados y creamos el marco de datos que necesitamos:

In [23]:
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

,dep,nom_commune,code_postal,coordonnees_gps,libelle_d_acheminement,code_commune_insee,ligne_5
0,77,CHEVRY COSSIGNY,77173.0,"[48.723292532, 2.6765377]",CHEVRY COSSIGNY,77114,COSSIGNY
1,77,DAMMARTIN SUR TIGEAUX,77163.0,"[48.808512927, 2.903759408]",DAMMARTIN SUR TIGEAUX,77154,NaN
2,77,NOISY RUDIGNON,77940.0,"[48.336585873, 2.933839851]",NOISY RUDIGNON,77338,NaN
3,77,ORLY SUR MORIN,77750.0,"[48.901173218, 3.226766301]",ORLY SUR MORIN,77345,NaN
4,77,OTHIS,77280.0,"[49.0826633, 2.655019992]",OTHIS,77349,NaN


4.3 Selección de características

Para nuestros dos conjuntos de datos, sólo necesitamos el municipio, el barrio, los códigos postales y las geolocalizaciones (latitud y longitud). Así que acabamos seleccionando las columnas que necesitamos:

In [24]:
df1 = wiki_london_data.drop( [ wiki_london_data.columns[0], wiki_london_data.columns[4], wiki_london_data.columns[5] ], axis=1)

In [26]:
df_2 = paris_field_data[['code_postal','nom_commune','libelle_d_acheminement','coordonnees_gps']]